# Genesis Demonstrator

In [4]:
import pandas as pd
import os
from typing import Final
from config import data_raw_folder, data_processed_folder
from timeeval import Datasets

In [5]:
dataset_collection_name = "Genesis"
source_folder = os.path.join(data_raw_folder, "genesis-demonstrator/data")
target_folder = data_processed_folder

from pathlib import Path
print(f"Looking for source datasets in {Path(source_folder).absolute()} and\nsaving processed datasets in {Path(target_folder).absolute()}")

Looking for source datasets in /home/projects/akita/data/benchmark-data/data-raw/genesis-demonstrator/data and
saving processed datasets in /home/projects/akita/data/benchmark-data/data-processed


## Dataset transformation and pre-processing

In [6]:
train_type = "unsupervised"
train_is_normal = False
input_type = "multivariate"
datetime_index = True
dataset_type = "real"

# create target directory
dataset_subfolder = os.path.join(input_type, dataset_collection_name)
target_subfolder = os.path.join(target_folder, dataset_subfolder)
try:
    os.makedirs(target_subfolder)
    print(f"Created directories {target_subfolder}")
except FileExistsError:
    print(f"Directories {target_subfolder} already exist")
    pass

dm = Datasets(target_folder)

Directories /home/projects/akita/data/benchmark-data/data-processed/multivariate/Genesis already exist


In [8]:
# get target filenames
dataset_name = "genesis-anomalies"
filename = f"{dataset_name}.test.csv"

source_file = os.path.join(source_folder, "Genesis_AnomalyLabels.csv")
path = os.path.join(dataset_subfolder, filename)
target_filepath = os.path.join(target_subfolder, filename)

# transform file
df = pd.read_csv(source_file)
#df = df.rename(columns={"Timestamp": "timestamp"})
df.insert(len(df.columns), "is_anomaly", df.loc[:, "Label"])
df.insert(1, "timestamp", pd.to_datetime(df["Timestamp"], unit='s'))
df = df.drop(columns=["Timestamp", "Label"])
df.to_csv(target_filepath, index=False)
print(f"Processed source dataset {source_file} -> {target_filepath}")

dataset_length = len(df)

# save metadata
dm.add_dataset((dataset_collection_name, dataset_name),
    train_path = None,
    test_path = path,
    dataset_type = dataset_type,
    datetime_index = datetime_index,
    split_at = None,
    train_type = train_type,
    train_is_normal = train_is_normal,
    input_type = input_type,
    dataset_length = dataset_length
)

dm.save()

Processed source dataset /home/projects/akita/data/benchmark-data/data-raw/genesis-demonstrator/data/Genesis_AnomalyLabels.csv -> /home/projects/akita/data/benchmark-data/data-processed/multivariate/Genesis/genesis-anomalies.test.csv


In [9]:
dm.refresh()
dm._df.loc["Genesis"]

,train_path,test_path,dataset_type,datetime_index,split_at,train_type,train_is_normal,input_type,length
dataset_name,,,,,,,,,
genesis-anomalies,NaN,multivariate/Genesis/genesis-anomalies.test.csv,real,True,NaN,unsupervised,False,multivariate,16220
